In [1]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

import numpy as np
import os
import datetime
# torch.set_printoptions(profile="full")
# np.set_printoptions(threshold=np.inf)


import numpy as np
import copy

torch.set_printoptions(profile="full")
np.set_printoptions(threshold=np.inf)


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 5
batch_size = 50

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [5]:
class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 25, kernel_size=3),
            torch.nn.BatchNorm2d(25),
            torch.nn.ReLU(inplace=True)
        )
 
        self.layer2 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
 
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(25, 50, kernel_size=3),
            torch.nn.BatchNorm2d(50),
            torch.nn.ReLU(inplace=True)
        )
 
        self.layer4 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
 
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(50 * 5 * 5, 2048),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(2048, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(128, 10)
        )
 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = MyNet().to(device)

print(model)

MyNet(
  (layer1): Sequential(
    (0): Conv2d(1, 25, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer2): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(25, 50, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer4): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=1250, out_features=2048, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=2048, out_features=1024, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=1024, out_features=128, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [6]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
# model.load_state_dict(torch.load("./myfed_normal_save/model18-32-05.pth")["state_dict"])
dir = os.listdir("./merge_save")
modelnames = []
for mt in dir:
    if(mt[0:len("merge_mnist_moreFC_")] == "merge_mnist_moreFC_"):
        modelnames.append(mt)
# print(modelnames)
modelset = []
for mn in modelnames:
    modelset.append([model.load_state_dict(torch.load("./merge_save/"+mn)["state_dict"]),mn])
# print(dir)
savepath = "./metric/vr_metric-mnist-more_layers="

In [8]:
len(modelset)

200

In [10]:
for mode, modelname in modelset:
    parm = {}
    for name,parameters in model.named_parameters():
        parm[name]=parameters
        # print(name)
    tensors = []
    model.eval()
    with torch.no_grad():

        t=5
        group1 = []
        group2 = []
        group3 = []
        group4 = []
        r = random.sample(range(1,batch_size - 1), t)
        # print(r)
        for i in range(0,t):
            

            for X_test, Y_test in test_loader:
                X_test = X_test.to(device)
                Y_test = Y_test.to(device)
            

            X_single_data = X_test[r[i]]
            Y_single_data = Y_test[r[i]]

            # print(X_single_data)
            
            layerP = model.layer1(torch.unsqueeze(X_single_data,dim=0))
            layerP = model.layer2(layerP)
            layerExtract1 = torch.nn.MaxPool2d(kernel_size=4, stride=2)(layerP)
            layerExtract1 = layerExtract1.squeeze().reshape(-1,)
            # print(layerExtract1.cpu().detach().numpy())
            group1.append(layerExtract1.cpu().detach().numpy())
            layerP = model.layer3(layerP)
            layerP = model.layer4(layerP)
            layerExtract2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)(layerP)
            layerExtract2 = layerExtract2.squeeze().reshape(-1,)
            # print(layerExtract2.shape)
            group2.append(layerExtract2.cpu().detach().numpy())
            layerP = layerP.view(layerP.size(0), -1)
            # print(layerP.shape)
            # layerP = model.fc(layerP)
            tensor1 = torch.mm(layerP, parm['fc.0.weight'].data.permute(1,0)) + parm['fc.0.bias']
            # print(tensor1.shape)
            tensor2 = torch.mm(tensor1, parm['fc.2.weight'].data.permute(1,0)) + parm['fc.2.bias']
            layerExtract3 = tensor2.squeeze().reshape(-1,)
            # print(layerExtract3.shape)
            group3.append(layerExtract3.cpu().detach().numpy())
            tensor3 = torch.mm(tensor2, parm['fc.4.weight'].data.permute(1,0)) + parm['fc.4.bias']
            layerExtract4 = tensor3.squeeze().reshape(-1,)
            # print(layerExtract4.shape)
            group4.append(layerExtract4.cpu().detach().numpy())
            # tensor4 = torch.mm(tensor3, parm['fc.6.weight'].data.permute(1,0)) + parm['fc.6.bias']
            
            # layerExtract2 = layerP.squeeze().reshape(-1,128)
            # # print(layerExtract2.shape)
            # group3.append(layerExtract2.cpu().detach().numpy()[0])
            # layerP = layerP.detach().squeeze().reshape(-1,512*4*4)

            # tensor1 = torch.mm(layerP, parm['module.fc14.weight'].data.permute(1,0)) + parm['module.fc14.bias']
            # tensor1 = torch.mm(tensor1, parm['module.fc15.weight'].data.permute(1,0)) + parm['module.fc15.bias']
            
            # group4.append(tensor1.cpu().detach().numpy()[0])

    l = 0
    for tensors in [group1, group2, group3, group4]:
        # print(len(tensors))
        # print(tensors)
        nb = np.linspace(0,0,tensors[0].shape[0])
        print(nb.shape)
        for i in tensors:
            nb += i
        nb /= t

        ns = np.linspace(0,0,tensors[0].shape[0])
        vr = np.linspace(0,0,tensors[0].shape[0] ** 2).reshape(tensors[0].shape[0],tensors[0].shape[0])

        for i in range(0,len(ns)):
            for nps in tensors:
                ns[i] += nps[i] ** 2
            ns[i] = ns[i] ** 0.5

        for i in range(0,len(ns)):
            # print(i)
            for j in range(i+1,len(ns)):
                for nps in tensors:
                    vr[i,j] += (nps[i] - nb[i]) * (nps[j] - nb[j])
                # vp[i,j] /= (ns[i] * ns[j])
                vr[i,j] = vr[i,j] / ((ns[i] * ns[j]) + int(not (ns[i] * ns[j])))
                # print(vr[i,j])

        dt = datetime.datetime.now()
        # np.savetxt("./misc/vr_metric"+dt.strftime("%d--%H-%M-%S")+".dat", vp+vp.T, fmt="%1.5f")
        # np.savetxt(savepath+modelname+str(l)+".dat", vr+vr.T, fmt="%1.5f")
        print(savepath+modelname+str(l)+".dat")
        l += 1

tensor([-1995460.8750, -3325825.5000, -1330490.7500, -1558905.5000,
        -2567230.0000, -2046265.8750, -1605028.5000, -1281144.2500,
         -984229.6250, -2429898.0000, -1582911.2500, -2364540.2500,
        -2086947.5000, -2329387.7500, -2006434.3750, -2115348.0000,
        -3335445.2500, -2048539.1250, -2266822.5000, -1189532.7500,
         -669804.2500, -2620455.0000,  -926684.9375,  -640917.6250,
        -2714945.5000, -1318190.3750, -1499811.6250, -2306400.5000,
        -2782722.5000, -2736740.7500, -1865440.6250, -1280716.7500,
        -2355278.2500, -2576197.2500, -1961165.3750, -1889534.5000,
        -1892506.5000, -1788064.5000, -1145415.3750, -1042733.7500,
        -1770762.3750, -2386319.7500, -1536253.5000, -2664260.0000,
        -1540128.6250, -1357294.0000, -2310819.7500, -2983730.0000,
        -1036441.8125, -2865972.7500, -1107280.3750, -3055051.0000,
        -1973501.6250, -2016636.6250, -1656225.7500, -1912293.7500,
        -1487096.3750, -1191876.6250,  -785092.9

KeyboardInterrupt: 

In [44]:
int(not -1)

0